In [1]:
import cv2, copy
import numpy as np
from metavision_core.event_io import EventsIterator

mv_iterator = EventsIterator(input_path='', delta_t=10000)# for 10 ms
orig_height, orig_width = mv_iterator.get_size()
print(f'Original input -> height, width: {orig_height}, {orig_width}')
# Original input -> height, width: 720, 1280



# Since managing the HD size of the input is expensive, we usually prefer to reduce the input size
scale_down_factor = 4
downScaled_height, downScaled_width = orig_height//scale_down_factor, orig_width//scale_down_factor
print(f'Down scaled input -> height, width: {downScaled_height}, {downScaled_width}')
# Down scaled input -> height, width: 180, 320

N_POL = 2 # number of polarities 

# Define the size of the region of interest (ROI)
ROI_WIDTH = 100
ROI_HEIGHT = 70
CENTER_Y, CENTER_X = downScaled_height//2, downScaled_width//2
ROI_Y0, ROI_X0, ROI_Y1, ROI_X1 = CENTER_Y-ROI_HEIGHT//2, CENTER_X-ROI_WIDTH//2, CENTER_Y+ROI_HEIGHT//2, CENTER_X+ROI_WIDTH//2



for i, evs in enumerate(mv_iterator): # to iterate over the EventsIterator object
    
    # to hold the events over a period of delta_t as an BGR (openCV format) frame
    play_back_frame_downScaled = np.zeros(( downScaled_height, downScaled_width, 3), dtype=np.uint8)
    play_back_frame_roi = np.zeros(( ROI_HEIGHT, ROI_WIDTH, 3), dtype=np.uint8)


    if (evs.size == 0):# or (i%2==0): # We can drop some iterations 
        pass
    else:
        #**************************** To use only positive (negative) events ****************************
        # evs = evs[evs['p']==1]

        #*************************** Use input events to create the downscaled frame ***************************
        # Here we are creating a CTI (Constant Time Interval) pseudo-frame. Unlike tonic (https://github.com/neuromorphs/tonic), 
        # we do not count the number of events emitted by each individual pixel durring the interval.
        # To create a CNE (Constant Number of Events) pseudo-frame, set n_events = (desired number of events per frame) 
        # when initializing the EventsIterator object.
        
        tmp_evs = copy.deepcopy(evs)

        # Downscaling the event spatially
        tmp_evs['x'] = tmp_evs['x']//scale_down_factor
        tmp_evs['y'] = tmp_evs['y']//scale_down_factor

        play_back_frame_downScaled[ tmp_evs[tmp_evs['p']==0]['y'], tmp_evs[tmp_evs['p']==0]['x'], 0] = 255 # put the negative events in B channel
        play_back_frame_downScaled[ tmp_evs[tmp_evs['p']==1]['y'], tmp_evs[tmp_evs['p']==1]['x'], 1] = 255 # put the positive events in G channel                                                                                                               # let the R channel as zero

        cv2.rectangle(play_back_frame_downScaled, (ROI_X0, ROI_Y0), (ROI_X1, ROI_Y1), (0, 0, 255),2)# For multi-box
        #**************************** Cropping the defined ROI over the downscaled events ****************************
        tmp_evs = tmp_evs[ ( (ROI_Y0<=tmp_evs['y'])&(tmp_evs['y']<ROI_Y1) ) & ( (ROI_X0<=tmp_evs['x'])&(tmp_evs['x']<ROI_X1) ) ]
        # Re-coordinate the events
        tmp_evs['y']-=ROI_Y0
        tmp_evs['x']-=ROI_X0
        play_back_frame_roi[ tmp_evs[tmp_evs['p']==0]['y'], tmp_evs[tmp_evs['p']==0]['x'], 0] = 255
        play_back_frame_roi[ tmp_evs[tmp_evs['p']==1]['y'], tmp_evs[tmp_evs['p']==1]['x'], 1] = 255


        cv2.imshow(f'Camera: Scale {scale_down_factor}:1',cv2.resize(play_back_frame_downScaled, (720, 400)))
        # cv2.imshow(f'Camera: Cropepd -> Scale  {scale_down_factor}:1',cv2.resize(play_back_frame_roi, (720, 400)))


        # Once the frames are available, we can apply image processing to perform various tasks
        # Here, we are detecting the contours of a certain size.
        # Detecting contours in the downscaled frames
        gray_image = cv2.cvtColor(play_back_frame_downScaled, cv2.COLOR_BGR2GRAY)
        _, thresholded_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Define the desired size of contours 
        max_contour_area_to_detect = (gray_image.shape[0]*gray_image.shape[1])//2
        min_contour_area_to_detect = (gray_image.shape[0]*gray_image.shape[1])//7

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            x_min, y_min = float('inf'), float('inf')
            x_max, y_max = float('-inf'), float('-inf')
            
            for i, contour in enumerate(contours):
                contour_area = cv2.contourArea(contour)
                if (contour_area>min_contour_area_to_detect) and (contour_area<max_contour_area_to_detect):

                    # Get the bounding box for each contour
                    x, y, w, h = cv2.boundingRect(contour)                
                    # Update the extreme points
                    x_min = min(x_min, x)
                    y_min = min(y_min, y)
                    x_max = max(x_max, x + w)
                    y_max = max(y_max, y + h)
                    cv2.rectangle(play_back_frame_downScaled, (x_min, y_min), (x_max, y_max), (0, 255, 255),2)# For multi-box
            # # For single box
            # if (contour_area>min_contour_area_to_detect) and (contour_area<max_contour_area_to_detect):
            #     cv2.rectangle(play_back_frame_downScaled, (x_min, y_min), (x_max, y_max), (0, 255, 255), 2)
        # Display the image with the rectangle with or without a contoursssss
            cv2.imshow(f'Camera: Scale {scale_down_factor}:1 --> Bounding Boxs', cv2.resize(play_back_frame_downScaled,  (720, 400)))
        
        # Detecting contours in the ROI
        gray_image = cv2.cvtColor(play_back_frame_roi, cv2.COLOR_BGR2GRAY)
        _, thresholded_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Define the desired size of contours 
        max_contour_area_to_detect = (gray_image.shape[0]*gray_image.shape[1])//2
        min_contour_area_to_detect = (gray_image.shape[0]*gray_image.shape[1])//7

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            x_min, y_min = float('inf'), float('inf')
            x_max, y_max = float('-inf'), float('-inf')
            
            for i, contour in enumerate(contours):
                contour_area = cv2.contourArea(contour)
                if (contour_area>min_contour_area_to_detect) and (contour_area<max_contour_area_to_detect):

                    # Get the bounding box for each contour
                    x, y, w, h = cv2.boundingRect(contour)                
                    # Update the extreme points
                    x_min = min(x_min, x)
                    y_min = min(y_min, y)
                    x_max = max(x_max, x + w)
                    y_max = max(y_max, y + h)
                    cv2.rectangle(play_back_frame_roi, (x_min, y_min), (x_max, y_max), (255, 255, 255),1)# For multi-box
            # # For single box
            # if (contour_area>min_contour_area_to_detect) and (contour_area<max_contour_area_to_detect):
            #     cv2.rectangle(play_back_frame_downScaled, (x_min, y_min), (x_max, y_max), (0, 255, 255), 2)
        # Display the image with the rectangle with or without a contoursssss
            cv2.imshow(f'Camera: Cropepd -> Scale  {scale_down_factor}:1 --> Bounding Boxs', cv2.resize(play_back_frame_roi,  (720, 400)))



        cv2.waitKey(1)  




Original input -> height, width: 720, 1280
Down scaled input -> height, width: 180, 320
